# Analiza Wielowymiarowa - zajecia 10 - Hierarchiczna analiza skupień

In [ ]:
from multidim.utils import resolve_stata, load_stata

STATA_PATH, STATA_TYPE = resolve_stata(version = 17, stype = "se")
# make sure they are proper ones
STATA_PATH, STATA_TYPE

In [ ]:
load_stata(STATA_PATH, STATA_TYPE)

In [ ]:
import pandas as pd
import scipy
import sklearn
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from multidim.funs import plot_dendrogram

### Przykład 1.

Dane i przykład zostały pożyczone z podręcznika  
Sophia Rabe-Hesketh i Brian Everitt  
"A Handobook of Statistical Analyses using Stata"  
Dane dotyczą czaszek ludzkich znalezionych w Tybecie

Zaladowanie danych

In [ ]:
from multidim.datasets import load_tibetan
tibetan = load_tibetan()
tibetan_copy = tibetan.copy()

Opis zbioru danych

In [ ]:
%%stata -d tibetan_copy
des
sum

In [ ]:
tibetan.describe().T

Zmienne sa mierzone w identycznych jednostkach (mm)
Nie ma potrzeby standaryzacji wartosci zmiennych


#### Metoda pojedynczego wiazania

In [ ]:
# %stata help cluster

In [ ]:
%%stata
cluster singlelinkage length breadth height upper face, name(pojedyncze)
cluster dendrogram

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "single")
model = model_base.fit(tibetan[["length", "breadth", "height", "upper", "face"]])
plt.title("Hierarchical Clustering Dendrogram - single")
# Default dendogram, looks quite similar to default STATA
plot_dendrogram(model)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

#### Metoda pelnego wiazania

In [ ]:
%%stata
cluster completelinkage length breadth height upper face, name(pelne)
cluster dendrogram

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "complete")
model = model_base.fit(tibetan[["length", "breadth", "height", "upper", "face"]])
plt.title("Hierarchical Clustering Dendrogram - complete")
# Default dendogram, looks quite similar to default STATA
plot_dendrogram(model)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

Ten dendrogram wskazuje na hierarchiczny charakter danych.
Dlugosc pionowych linii (wysokosc) pokazuje roznice miedzy skupieniami, 
im linie sa dluzsze tym obiekty bardziej roznia sie

#### Metoda przecietnego wiazania

In [ ]:
%%stata
cluster averagelinkage length breadth height upper face, name(przecietne)
cluster dendrogram

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "average")
model = model_base.fit(tibetan[["length", "breadth", "height", "upper", "face"]])
plt.title("Hierarchical Clustering Dendrogram - average")
# Default dendogram, looks quite similar to default STATA
plot_dendrogram(model)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

#### Metoda medianowego wiazania

In [ ]:
%%stata
cluster medianlinkage length breadth height upper face, name(medianowe)

In [ ]:
%stata cluster list

#### Metoda Warda

In [ ]:
%%stata
cluster wardslinkage length breadth height upper face, name(Ward)
cluster dendrogram

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "ward")
model = model_base.fit(tibetan[["length", "breadth", "height", "upper", "face"]])
plt.title("Hierarchical Clustering Dendrogram - ward")
# Default dendogram, looks quite similar to default STATA
plot_dendrogram(model)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

Ten dendrogram wskazuje na hierarchiczny charakter danych.
Prosze zwrocic uwage ze metoda Warda wykorzystuje inna miare odleglosci

##### Kryterium wyboru optymalnej liczby grup

**Brak kodu python w tej czesci**

In [ ]:
%%stata
cluster stop pojedyncze, rule(duda)
cluster stop pojedyncze, rule(calinski)

Dendrogram dla metody pojedynczego wiazania

In [ ]:
%stata cluster dendrogram pojedyncze 

Dendrogram dla metody pojedynczego wiazania ucięty po 6 wezlach, opcja "showcount" pokazuję liczbę liści w gałęzi

In [ ]:
%stata cluster dendrogram pojedyncze, cutn(6) showcount

In [ ]:
%%stata
cluster stop pelne, rule(duda)
cluster stop pelne, rule(calinski)

Dendrogram dla metody pelnego wiazania

In [ ]:
%stata cluster dendrogram pelne

Cendrogram dla metody pelnego wiazania ucięty po 3 wezlach

In [ ]:
%stata cluster dendrogram pelne, cutn(3) showcount

In [ ]:
%%stata
cluster stop przecietne, rule(duda)
cluster stop przecietne, rule(calinski)

In [ ]:
%%stata
cluster stop medianowe, rule(duda)
cluster stop medianowe, rule(calinski)

In [ ]:
%%stata
cluster stop Ward, rule(duda)
cluster stop Ward, rule(calinski)

Chcemy zobaczyc charakterystyki grup czaszek uzyskanych metoda pelnego wiazania.
Tworzymy identyfikatory grup.

In [ ]:
%stata cluster generate grupa = groups(3), name(pelne)

Tablica (tabela) liczebnosci

In [ ]:
%stata tabulate grupa

### Przykład 2

W tym przykladzie wykorzystamy znane juz Panstwu dane dotyczace trzech odmian irysow.
Cechy kwiatów to:  
- Dlugosc platka [cm] (petal lenght)
- Szerokosc platka [cm] (petal width)
- Dlugosc listka kielicha [cm]
- Szerokosc listka kielicha [cm]


Wczytanie zbioru danych

In [ ]:
from multidim.datasets import load_iris

In [ ]:
iris = load_iris()
iris.columns = ["seplen", "sepwid", "petlen", "petwid", "iris"]
iris_copy = iris.copy()
iris_x = iris.iloc[:,:-1]
iris_y = iris.iloc[:,-1]

Opis zbioru danych

In [ ]:
%stata clear

In [ ]:
%%stata -d iris_copy
des

Podstawowe statystyki opisowe

In [ ]:
%%stata
bysort iris: su seplen sepwid petlen petwid 

In [ ]:
iris_x.groupby(iris_y).describe().T

Podział metodą pojedynczego wiazania

In [ ]:
%%stata
cluster singlelinkage seplen sepwid petlen petwid , name(pojedyncze)

In [ ]:
# Only one of distance_threshold and n_clusters has to be specified
model_base = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "single")
model = model_base.fit(iris_x)

Dla zbioru o wiekszej liczbie obserwacji nie mozna wyswietlic pelnego dendrogramu  
przydatne sa opcje:

1. cutnumber()           wyswietla okreslona liczbe galezi  
2. cutvalue()            wyswietla tylko rozniace sie galezie o zadana wartosc  

opcja showcount pokazuje liczbe obserwacji w kazdej galezi

In [ ]:
%stata cluster dendrogram, cutnumber(10) showcount

In [ ]:
%stata cluster dendrogram, cutvalue(0.8) showcount

W przypadku python-owej funkcji `scipy.cluster.hierarchy.dendrogram` mamy dwie inne opcje:

1. lastp - Ostatnie p klastrów niesingletonowych utworzonych w powiązaniu; Powinna dawac wyniki takie jak cutnumber w stata.
2. level - Wyświetlanych jest nie więcej niż p poziomów drzewa dendrogramu.

Source: [URL](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html#scipy.cluster.hierarchy.dendrogram)

In [ ]:
plot_dendrogram(model, truncate_mode = 'lastp', p = 10)


In [ ]:
plot_dendrogram(model, truncate_mode = 'level', p = 2)


Tworzymy identyfikator 3 grup

In [ ]:
%stata cluster generate gr_single = groups(3), name(pojedyncze)

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=None, n_clusters=3, linkage = "single")
model = model_base.fit(iris_x)

Czy klasyfikacja jest poprawna?

In [ ]:
%stata tab iris gr_single

In [ ]:
pd.crosstab(iris_y, model.labels_)

Sprobujmy przeprowadzadzić analize inna metoda

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "complete")
model = model_base.fit(iris_x)

In [ ]:
%%stata
cluster completelinkage seplen sepwid petlen petwid , name(pelne)
cluster dendrogram, cutnumber(10)
cluster dendrogram, cutvalue(0.5)

In [ ]:
plot_dendrogram(model, truncate_mode = 'lastp', p = 10)


In [ ]:
plot_dendrogram(model, truncate_mode = 'level', p = 2)

Tworzymy identyfikator 3 grup

In [ ]:
%stata cluster generate gr_complete = groups(3), name(pelne)

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=None, n_clusters=3, linkage = "complete")
model = model_base.fit(iris_x)

Czy klasyfikacja jest poprawna? Porówujemy klasyfikację z danymi

In [ ]:
%stata tab iris gr_complete

In [ ]:
pd.crosstab(iris_y, model.labels_)

Sprobujmy przeprowadzadzić analize kolejna metoda

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=0, n_clusters=None, linkage = "ward")
model = model_base.fit(iris_x)

In [ ]:
%%stata
cluster wardslinkage seplen sepwid petlen petwid , name(ward)
cluster dendrogram, cutnumber(10)

In [ ]:
plot_dendrogram(model, truncate_mode = 'lastp', p = 10)

Tworzymy identyfikator 3 grup

In [ ]:
model_base = AgglomerativeClustering(distance_threshold=None, n_clusters=3, linkage = "ward")
model = model_base.fit(iris_x)

In [ ]:
%stata cluster generate gr_ward = groups(3), name(ward)

Sprawdzamy czy klasyfikacja jest poprawna? Porównujemmy klasyfikację z danymi

In [ ]:
%stata tab iris gr_ward

In [ ]:
pd.crosstab(iris_y, model.labels_)